# Parsing

This isn't so much a useful nmigen tool in so much as it's useful for parsing or assembling arbitrary binary packets, which is naturally required to build a digital radio.

Perhaps in the future this can be expanded to a hardware-based parser.

First starting with imports...

In [16]:
from pprint import pprint

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from nmigen import Module, Memory

from alldigitalradio.parsing import Chunk, chunk, flip, num, Format, o, lsb_num
from alldigitalradio.util import pack_mem

I commonly want to create a test packet, but I don't want to define a million custom structs on the fly or just manually type out a bunch of ones and zeros.

For this, I use `Chunk` as well as other convenient methods to insert bytes and flip for various endianness. This is an example for a bluetooth packet:

In [9]:
packet = Chunk(
    preamble=[0, 1, 0, 1, 0, 1, 0, 1],
    access_address=flip(o(0x8e, 0x89, 0xbe, 0xd6)),

    ble_packet=Chunk(
      pdu_type=flip([0, 0, 1, 0]), # Connectible undirected advertising event (BLE advertising)
      rfu_0=[0]*2, # Reserved for future use
      tx_add=[0],
      rx_add=[0],
      length=[0]*8, # Filled in later

      payload=Chunk(
        device_addr=flip(o(0x90, 0xd7, 0xeb, 0xb1, 0x92, 0x99)),
        payload_1=Chunk(
          length=flip(o(0x02)), 
          kind=flip(o(0x01)), # Flag
          data=flip(o(0x05))), # Limited discoverable, BR/EDR not supported
        payload_2=Chunk(
            length=flip(o(1 + len("I LOVE MINDY"))),
            kind=flip(o(0x08)), # Short name
            name=o(*[ord(c) for c in "I LOVE MINDY"], lsb=True))),
    ),

    crc=[0]*24
)

To parse this, you can use `Format` to do the reverse. The numbers here are the number of bits in a given section.

In [18]:
# First define a format, which is a nested list of sections N bits wide
format = Format(
    preamble=8,
    access_address=32,
    ble_packet=Format(
        pdu_type=4,
        rfu_0=2,
        tx_add=1,
        rx_add=1,
        length=8,
        payload=Format(
            device_addr=48, # The sending device
            payload_1=Format(
              length=8, 
              kind=8,
              payload=8),
            payload_2=Format(
              length=8,
              kind=8,
              payload=12*8)
        ),
    ),
    crc=24
)

# Take the bits from above and parse them again
bits = packet.bits()
read, parsed = format.parse(bits)

# You can easily index into the parsed packages as if it was a JSON-style object
print(''.join(map(chr, pack_mem(parsed.ble_packet.payload.payload_2.payload, 8))))

# Or just print everything out for inspection
pprint(parsed.json(), compact=True)

I LOVE MINDY
OrderedDict([('preamble', [0, 1, 0, 1, 0, 1, 0, 1]),
             ('access_address',
              [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
               0, 1, 0, 1, 1, 1, 0, 0, 0, 1]),
             ('ble_packet',
              OrderedDict([('pdu_type', [0, 1, 0, 0]), ('rfu_0', [0, 0]),
                           ('tx_add', [0]), ('rx_add', [0]),
                           ('length', [0, 0, 0, 0, 0, 0, 0, 0]),
                           ('payload',
                            OrderedDict([('device_addr',
                                          [1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
                                           1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
                                           1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
                                           1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
                                           1]),
                                         ('payload_1',
                                  

Note that parsing variable length or repeated fields isn't supported as I have yet to figure out a convenient and simple syntax to define them.